# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [23]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [26]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-17-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-08-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-02-events.csv', '/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-22-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [27]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_new csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [28]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra

#### Creating a Cluster

In [29]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [30]:
# Create KEYSPACE

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [31]:
# Set KEYSPACE to the keyspace specified above

try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


### Task 1: Return the artist, song title and song's length in the music app history using search of sessionID and the item in that session (item_in_session).

In [32]:
#the csv.reader function below pulls everything in as a string, not a number. so i will have to cast these to right type in the insert line
#Creating table for this query here
query = "CREATE TABLE IF NOT EXISTS song_library "
query = query + "(session_id INT, item_in_session INT, artist_name TEXT, song_name TEXT, song_length FLOAT, PRIMARY KEY (session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [33]:
# read file data and insert into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_library (session_id, item_in_session, artist_name, song_name, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        #need to cast to correct type
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [34]:
#query to check if data was entered correctly by outputting expected results
#test using session_id = 338 and item_in_session = 4
query = "SELECT artist_name, song_name, song_length FROM song_library WHERE session_id = 338 AND item_in_session = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_name, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Task 2: Return the name of artist, song (sorted by itemInSession) and user (first and last name), using search by userID and sessionID

In [35]:
#create table for second query

query = "CREATE TABLE IF NOT EXISTS artist_library "
query = query + "(user_id INT, session_id INT, artist_name TEXT, song_name TEXT, item_in_session INT,  user_firstname TEXT, user_lastname TEXT, \
                    PRIMARY KEY ((user_id, session_id), item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [36]:
# read file data and insert into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO artist_library (artist_name, song_name, session_id, item_in_session, user_id, user_firstname, user_lastname)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
         #need to cast to correct type
        session.execute(query, (line[0], line[9], int(line[8]), int(line[3]), int(line[10]), line[1], line[4]))

In [37]:
#query to check if data was entered correctly by outputting expected results
#test using uer_id = 10 and session_id = 182
query = "SELECT artist_name, song_name, user_firstname, user_lastname FROM artist_library where user_id = 10 AND session_id = 182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_name, row.user_firstname, row.user_lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Task 3: Return every user (first and last name) in the music app history, search on song title

In [38]:
query = "CREATE TABLE IF NOT EXISTS users_by_song "
query = query + "(song_name TEXT, user_id INT, user_firstname TEXT, user_lastname TEXT,  PRIMARY KEY ((song_name), user_id))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [39]:
# read file data and insert into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO users_by_song (song_name, user_id, user_firstname, user_lastname)"
        query = query + " VALUES (%s, %s, %s, %s)"
         #need to cast to correct type
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [40]:
#query to check if data was entered correctly by outputting expected results
#test using song_name = 'All Hands Against His Own'
query = "SELECT user_firstname, user_lastname, song_name FROM users_by_song WHERE song_name = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_firstname, row.user_lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [41]:
## Drop the table before closing out the sessions

#table 1 
query = "drop table song_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

#table 2
query = "drop table artist_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
#table 3
query = "drop table users"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="unconfigured table users"


### Close the session and cluster connection¶

In [42]:
session.shutdown()
cluster.shutdown()